## Ground-state preparation for TF Ising Model using QETU

Based on methodology from [arXiv:2204.05955](https://arxiv.org/abs/2204.05955)

The Hamiltonian that will be implemented is given by the following equation:

$$H_{\mathrm{TFIM}}=\underbrace{-\sum_{j=1}^{n-1}Z_{j}Z_{j+1}}_{H_{\mathrm{TFIM}}^{(1)}}\underbrace{-g\sum_{j=1}^{n}X_{j}}_{H_{\mathrm{TFIM}}^{(2)}}.$$
The Trotterized implementation of the Hamiltonian typically includes controlled operations for Hamiltonian components. In this case, the implementation is modified in such a way as to avoid this by implementing Pauli strings $K$. The strings are selected to anticommute with each grouped Hamiltonian term. In the TFIM case we can identify one String to commute with all terms:
$$K:=Y_1\otimes Z_2\otimes Y_3\otimes Z_4\otimes\cdots . $$
This can be implemented on cirq using the following function:

In [136]:
import cirq as cirq
import numpy as np
import cvxpy as cp

In [137]:
def K(n_qubits):
    if n_qubits % 2:
          raise Exception("Sorry, numbers of qubits must be even") 

    q = cirq.LineQubit.range(n_qubits)
    seq = []
    for i in range(int(n_qubits/2)):
        seq.append([cirq.Y(q[i]),cirq.Z(q[i+1])])
    return cirq.PauliString(seq)


Alongside our only Pauli string we must define out evolution operator $W$.

In [138]:
def rzz(rads,q,i):
    """Returns a gate with the matrix exp(-i Z⊗Z rads)."""
    return cirq.ZZPowGate(exponent=2 * rads / np.pi, global_shift=-0.5,).on(q[i],q[i+1])

g = 4
def W(n_qubits,tau):
    q = cirq.LineQubit.range(n_qubits)
    W = cirq.Circuit()
    # H_1:
    for i in range(n_qubits-1):
        W.append(rzz(tau,q,i))
    # H_2:
    for i in range(n_qubits):
        W.append(cirq.rx(tau).on(q[i]))
    
    return W
W(4,0.5)

0: ───ZZ─────────Rx(0.159π)─────────────────────────────
      │
1: ───ZZ^0.318───ZZ───────────Rx(0.159π)────────────────
                 │
2: ──────────────ZZ^0.318─────ZZ───────────Rx(0.159π)───
                              │
3: ───────────────────────────ZZ^0.318─────Rx(0.159π)───

In [139]:
def KWK_step(n_qubits,tau):
    q = cirq.LineQubit.range(n_qubits+1) #last qubit selected as ancilla
    KWK_step = cirq.Circuit()
    KWK_step.append(K(n_qubits).controlled_by(q[-1],control_values=[0]))
    KWK_step.append(W(n_qubits,tau))
    KWK_step.append(K(n_qubits).controlled_by(q[-1],control_values=[0]))
    return KWK_step

KWK_step(4,0.5)

0: ───PauliString(-iY)───ZZ─────────Rx(0.159π)─────────────────────────────PauliString(-iY)───
      │                  │                                                 │
1: ───X──────────────────ZZ^0.318───ZZ───────────Rx(0.159π)────────────────X──────────────────
      │                             │                                      │
2: ───Z─────────────────────────────ZZ^0.318─────ZZ───────────Rx(0.159π)───Z──────────────────
      │                                          │                         │
3: ───┼──────────────────────────────────────────ZZ^0.318─────Rx(0.159π)───┼──────────────────
      │                                                                    │
4: ───(0)──────────────────────────────────────────────────────────────────(0)────────────────

The complete circuit intersperses KWK operations with $e^{i\phi_i X}$ terms, where $\phi_i$ refers to adjustable phase factor parameters from the polynomial approximation of the shifted sign function. Thus we need to construct the polynomials via optimization. In this case the method used is Convex Optimization. We start off by witing out our $d$ degree target polynomial as a LC of Chebyshev polynomials $T_{2k}$:
$$F(x) = \sum_{k=0}^{d/2} T_{2k}(x) c_k $$
with unknown parameters $c_k$. Then we generate a matrix of coefficients $A_{jk}$ by discretizing the domain over Chevbyshev root values.



In [ ]:
def shifted_sign_function(x,mu):
    func = abs(np.heaviside(x-mu*np.ones(len(x)),0)-1)
    return func

def sigma(string,mu,delta,eta):
    if string == '+':
        return np.cos((mu-delta/2)/2)
    if string == '-':
        return np.cos((mu+delta/2)/2)
    if string == 'min':
        return np.cos((np.pi-eta)/2)
    if string == 'max':
        return np.cos(eta/2)
    else: 
        raise 'Invalid string input'
    

def target(coefs,mu,delta,eta,M):
    coefs_even = []
    for item in coefs: #we only want even Chebyshev polynomials
        coefs_even.append(item)
        coefs_even.append(0)

    #we want to discretize on the roots (M values)
    j = np.linspace(0,M-1,M) 
    x_j = -np.cos(j*np.pi/(M-1))
    F_xj = np.polynomial.chebyshev.chebval(x_j,coefs_even) #default domain is [-1,1] which is what we need

    # F_xj_abs = []
    # for F in F_xj:
    #     F_xj_abs = cp.hstack(F_xj_abs,cp.abs(F))
    norm_max = np.max(F_xj)

    c = 0.999

    T1 = []
    T2 = []
    sigma_plus = sigma('+', mu, delta, eta)
    sigma_max = sigma('max', mu, delta, eta)
    sigma_minus = sigma('-', mu, delta, eta)
    sigma_min = sigma('min', mu, delta, eta)

    for i in range(len(x_j)):
        x = x_j[i]
        if x <= sigma_plus and x >= sigma_max:
            T1.append(np.abs(F_xj[i]-c))
        if x_j[i] <= sigma_minus and x_j[i] >= sigma_min:
            T2.append(np.abs(F_xj[i]))

    max_T1 = np.max(T1)
    max_T2 = np.max(T2)
    target = np.max([max_T1,max_T2])
    return target, norm_max


In [ ]:
d = 20
mu = 1.0
eta = 0.1
delta = 0.4
M = 400
c = 0.999
# coefs = [0,0.1,0.2,-0.3,0.4,-0.5]
# print(target(coefs, mu, delta, eta, M)[0])
coefs = cp.Variable(d)
objective = cp.Minimize(target(coefs, mu, delta, eta, M)[0])
constraint = [target(coefs, mu, delta, eta, M)[1] <= c]
prob = cp.Problem(objective, constraint)




In [179]:
result = prob.solve()
print(coefs.value)

None
